<a href="https://colab.research.google.com/github/sbanwaskar/CS-579-course-project/blob/main/Classifications_diff_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import pickle
import numpy as np
import pandas as pd
import scipy.stats as st
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


import numpy as np
from sklearn.impute import SimpleImputer 
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# #data import
# from google.colab import files
# uploaded = files.upload()
!wget https://www.dropbox.com/s/xkozscpcryu871h/labels_1540_4classes_icmla_21.pck
!wget https://www.dropbox.com/s/ovt8g99wjcz1mvr/mvts_1540_icmla_21.pck


In [ ]:
#reading pickle files
def load(file_name):
    with open(file_name, 'rb') as fp:
        obj = pickle.load(fp)
    return obj


Sampled_inputs=load("mvts_1540_icmla_21.pck")


Sampled_labels=load("labels_1540_4classes_icmla_21.pck")    

temp=Sampled_inputs[0]
print(temp)
df = pd.DataFrame(temp)
trainData=Sampled_inputs
trainLabel=Sampled_labels
print("trainData.shape: ",trainData.shape)
print("trainLebel.shape: ",trainLabel.shape)

In [ ]:
# #standardization/z normalization of the univaraite time series
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# npArrays=[]
# for l in range(0, len(trainData)):
#   trainData_std = sc.fit_transform(trainData[l])
#   #trainData_std = trainData_std.astype(np.float64)
#   #print(type(trainData_std[0][0]))
#   npArrays.append(trainData_std)

# print(type(npArrays))
# arr = np.asarray(npArrays)
# print(type(arr))
# trainData=arr
# print("trainData.shape: ",trainData.shape)
# print(type(trainData))
# print("trainLebel.shape: ",trainLabel.shape)
# print(type(trainLabel))

Saving all data to csv (different files)

In [ ]:
def time_series(trainData):

  rowList = []
#Here we parse through the entire trainData matrix and append eveything to the row list.
  for i in range(0, len(trainData)):

    for j in range(0, len(trainData[i])):
      
      rowList.append(trainData[i][j])
  # print(len(rowList))
  return rowList

def time_series1(rowList):
  rows, cols = (len(rowList), len(rowList[0])-1)

# here we declare a row list to which we append the values for all the 8 parameters.
  rowList1 = [[0 for i in range(cols)] for j in range(rows)]
#Here we parse through the entire trainData matrix and append eveything to the row list.
  for i in range (0, len(rowList)):

    for j in range(0,len(rowList[i])-1):

      rowList1[i][j] = rowList[i][j+1] - rowList[i][j]
  return rowList1

def get_mean_std(rowList):
  Mean = np.array(rowList).mean(1)
  means = np.reshape(Mean, (1540, 33))
  standardDeviation = np.array(rowList1).std(1)
  stdDev = np.reshape(standardDeviation, (1540, 33))
  return means,stdDev

def get_skew_and_kurtosis(rowList):
  df = pd.DataFrame(rowList)
  skew_df = df.skew(1)
  kurtosis_df = df.kurtosis(1)
  skew_rs = np.reshape(skew_df.values,(1540, 33))
  kurtosis_rs = np.reshape(kurtosis_df.values,(1540, 33))
  return skew_rs , kurtosis_rs

def get_all_params(rowList):
  mean , std_dev =get_mean_std(rowList)
  skew , kurtosis = get_skew_and_kurtosis(rowList)
  return mean,std_dev, skew,kurtosis

rowList = time_series(trainData)
rowList1 = time_series1(rowList)

mean,std_dev,skew , kurtosis = get_all_params(rowList)
mean1,std_dev1,skew1 , kurtosis1 = get_all_params(rowList1)

In [ ]:
all_params = [mean, std_dev,skew , kurtosis,mean1,std_dev1,skew1 , kurtosis1]
all_param_names = ['mean','std_dev','skew','kurtosis','mean1','std_dev1','skew1','kurtosis1']


## Method 1 : All Data in one csv please follow all_param_names to check the sequencing of data
# write
all_arrays_list = [a for a in all_params]
all_arrays_np = np.array(all_arrays_list).reshape(1540*8 , 33)
np.savetxt("all_params_data.csv", all_arrays_np, delimiter=",")
# read
data_read_back = np.loadtxt('all_params_data.csv',delimiter=',')
all_param_names = ['mean','std_dev','skew','kurtosis','mean1','std_dev1','skew1','kurtosis1']
all_param_data = data_read_back.reshape(8, 1540 , 33)


## Method 2 : All in one excel with different sheets (more organized, better when checking for issues in data by reading the excel manually)
# write-only
df_list = [pd.DataFrame(val) for val in all_params] 
filename = 'all_params_excel'
writer = pd.ExcelWriter(filename+'.xlsx')
for i, df in enumerate(df_list):
    df.to_excel(writer,sheet_name=all_param_names[i])
writer.save() 

In [ ]:
# All in one compressed pickle

final_data = {'mean':mean,
              'std_dev':std_dev,
              'skew':skew,
              'kurtosis':kurtosis,
              'mean1':mean1,
              'std_dev1':std_dev1,
              'skew1':skew1,
              'kurtosis1':kurtosis1}

import bz2
import pickle
import _pickle as cPickle

filename = 'all_data_in_one'

def decompress_pickle(file):
  data = bz2.BZ2File(file, 'rb')
  data = cPickle.load(data)
  return data

def compressed_pickle(title, data):
  with bz2.BZ2File(title + '.pbz2', 'w') as f: 
   cPickle.dump(data, f)

compressed_pickle(filename, final_data) 
data = decompress_pickle(filename+'.pbz2') 

In [ ]:
!zip -r all_data_in_one.zip all_*
from google.colab import files
files.download('all_data_in_one.zip') 

In [ ]:
def time_series():

  rowList = []
#Here we parse through the entire trainData matrix and append eveything to the row list.
  for i in range(0, len(trainData)):

    for j in range(0, len(trainData[i])):
      
      rowList.append(trainData[i][j])
  # print(len(rowList))
  return rowList

def time_series1(rowList):
  rows, cols = (len(rowList), len(rowList[0])-1)

# here we declare a row list to which we append the values for all the 8 parameters.
  rowList1 = [[0 for i in range(cols)] for j in range(rows)]
#Here we parse through the entire trainData matrix and append eveything to the row list.
  for i in range (0, len(rowList)):

    for j in range(0,len(rowList[i])-1):

      rowList1[i][j] = rowList[i][j+1] - rowList[i][j]
  return rowList1

In [ ]:
def mean():
  Mean = np.array(rowList).mean(1)
  means = np.reshape(Mean, (1540, 33))
  df = pd.DataFrame(means)
  print(means)
  df.to_csv("mean.csv")
def deviation():
  standardDeviation = np.array(rowList).std(1)
  stdDev = np.reshape(standardDeviation, (1540, 33))
  DF = pd.DataFrame(stdDev)
  print("info for deviation")
  print(DF.info)
  DF.to_csv("standardDeviation.csv")
def  skew():
  df = pd.DataFrame(rowList)
  skewValue = np.array(df.skew(1))
  skews = np.reshape(skewValue, (1540, 33))
  df = pd.DataFrame(skews)
  print("info for skew")
  print(df.info)
  df.to_csv("skew.csv")
def kurt():
  df = pd.DataFrame(rowList)
  kurtosisValue = np.array(df.kurt(1))
  kurts = np.reshape(kurtosisValue, (1540, 33))
  df = pd.DataFrame(kurts)
  print("info for kurtsosis")
  print(df.info)
  df.to_csv("kurt.csv")



In [ ]:
def mean():
  Mean = np.array(rowList).mean(1)
  means = np.reshape(Mean, (1540, 33))
  df = pd.DataFrame(means)
  print(means)
  df.to_csv("mean.csv")
def deviation():
  standardDeviation = np.array(rowList).std(1)
  stdDev = np.reshape(standardDeviation, (1540, 33))
  DF = pd.DataFrame(stdDev)
  print("info for deviation")
  print(DF.info)
  DF.to_csv("standardDeviation.csv")
def  skew():
  df = pd.DataFrame(rowList)
  skewValue = np.array(df.skew(1))
  skews = np.reshape(skewValue, (1540, 33))
  df = pd.DataFrame(skews)
  print("info for skew")
  print(df.info)
  df.to_csv("skew.csv")
def kurt():
  df = pd.DataFrame(rowList)
  kurtosisValue = np.array(df.kurt(1))
  kurts = np.reshape(kurtosisValue, (1540, 33))
  df = pd.DataFrame(kurts)
  print("info for kurtsosis")
  print(df.info)
  df.to_csv("kurt.csv")



In [ ]:
def time_series1(rowList):
  rows, cols = (len(rowList), len(rowList[0])-1)

# here we declare a row list to which we append the values for all the 8 parameters.
  rowList1 = [[0 for i in range(cols)] for j in range(rows)]
#Here we parse through the entire trainData matrix and append eveything to the row list.
  for i in range (0, len(rowList)):

    for j in range(0,len(rowList[i])-1):

      rowList1[i][j] = rowList[i][j+1] - rowList[i][j]

  return rowList1
    #print(len(rowList))
# the size of the row list is 1540*33 = 50820.


      






  


In [ ]:
Mean = np.array(rowList1).mean(1)
means = np.reshape(Mean, (1540, 33))
df = pd.DataFrame(means)
df.to_csv("mean.csv")

In [ ]:
def mean1():
  Mean = np.array(rowList1).mean(1)
  means = np.reshape(Mean, (1540, 33))
  df = pd.DataFrame(means)
  df.to_csv("mean1.csv")
def deviation1():
  standardDeviation = np.array(rowList1).std(1)
  stdDev = np.reshape(standardDeviation, (1540, 33))
  DF = pd.DataFrame(stdDev)
  DF.to_csv("standardDeviation1.csv")
def skew1():
  df = pd.DataFrame(rowList1)
  skewValue = np.array(df.skew(1))

#print(len(skewValue))
  skews = np.reshape(skewValue, (1540, 33))
  df = pd.DataFrame(skews)
  df.to_csv("skew1.csv")
def kurt1():
  df = pd.DataFrame(rowList1)
  kurtosisValue = np.array(df.kurt(1))
#print(len(skewVale))
  kurts = np.reshape(kurtosisValue, (1540, 33))
  df = pd.DataFrame(kurts)
  df.to_csv("kurt1.csv")
#size of kurtosis is 1540*33



In [ ]:
def main_mean():
 df = pd.read_csv("all_params_data.csv")

In [ ]:
main_mean()

In [ ]:
from scipy.stats import loguniform
from pandas import read_csv
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#Here change the name of the file in [''] to check accuracy of required file
	# ['mean'] ['std_dev'] ['skew'] ['kurtosis'] ['mean1'] ['std_dev1'] ['skew1'] ['kurtosis']
X = decompress_pickle('all_data_in_one.pbz2')['kurtosis1']
Y = trainLabel
X.shape,Y.shape

((1540, 33), (1540,))

**KNN Classification**
**/SVM**





In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier

In [ ]:
 from sklearn.model_selection import train_test_split
 X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30)
 from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30)


In [ ]:
#using polynomial kernel
from sklearn.svm import SVC
svclassifier_polynomail = SVC(kernel='poly', degree=8)
svclassifier_polynomail.fit(X_train, Y_train)
#Y_pred = svclassifier.predict(X_test)
Y_pred = svclassifier_polynomail.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

In [ ]:
 #using gaussian rbf kernel
svclassifier_gauss = SVC(kernel='rbf',random_state=0)
svclassifier_gauss.fit(X_train, Y_train)
Y_pred = svclassifier_gauss.predict(X_test)
print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))


In [ ]:
#using sigmoid kernel

svclassifier_sigmoid = SVC(kernel='sigmoid',random_state=0)
svclassifier_sigmoid.fit(X_train, Y_train)
Y_pred = svclassifier_sigmoid.predict(X_test)
print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))


In [ ]:
#using linear kernel
svclassifier_linear = SVC(kernel='linear')
svclassifier_linear.fit(X_train, Y_train)
Y_pred = svclassifier_linear.predict(X_test)
print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))


In [ ]:

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
#from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Function importing Dataset

# Function to split the dataset
def splitdataset(balance_data):

	# Separating the target variable
	#Here change the name of the file in [''] to check accuracy of required file
	# ['mean'] ['std_dev'] ['skew'] ['kurtosis'] ['mean1'] ['std_dev1'] ['skew1'] ['kurtosis']
	X = decompress_pickle('all_data_in_one.pbz2')['std_dev1']
	Y = trainLabel

	# Splitting the dataset into train and test
	X_train, X_test, y_train, y_test = train_test_split(
	X, Y, test_size = 0.3, random_state = 100)
	
	return X, Y, X_train, X_test, y_train, y_test
	
# Function to perform training with giniIndex.
def train_using_gini(X_train, X_test, y_train):

	# Creating the classifier object
	clf_gini = DecisionTreeClassifier(criterion = "gini",
			random_state = 100,max_depth=3, min_samples_leaf=5)

	# Performing training
	clf_gini.fit(X_train, y_train)
	return clf_gini
	
# Function to perform training with entropy.
def tarin_using_entropy(X_train, X_test, y_train):

	# Decision tree with entropy
	clf_entropy = DecisionTreeClassifier(
			criterion = "entropy", random_state = 100,
			max_depth = 3, min_samples_leaf = 5)

	# Performing training
	clf_entropy.fit(X_train, y_train)
	return clf_entropy


# Function to make predictions
def prediction(X_test, clf_object):

	# Predicton on test with giniIndex
	y_pred = clf_object.predict(X_test)
	print("Predicted values:")
	print(y_pred)
	return y_pred
	
# Function to calculate accuracy
def cal_accuracy(y_test, y_pred):
	
	print("Confusion Matrix: ",
		confusion_matrix(y_test, y_pred))
	
	print ("Accuracy : ",
	accuracy_score(y_test,y_pred)*100)
	
	print("Report : ",
	classification_report(y_test, y_pred))


def main():
	
  #Here change the name of the file in [''] to check accuracy of required file
	# ['mean'] ['std_dev'] ['skew'] ['kurtosis'] ['mean1'] ['std_dev1'] ['skew1'] ['kurtosis']
	data = decompress_pickle('all_data_in_one.pbz2')['std_dev1']
	X, Y, X_train, X_test, y_train, y_test = splitdataset(data)
	clf_gini = train_using_gini(X_train, X_test, y_train)
	clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
	
	# Operational Phase
	print("Results Using Gini Index:")
	
	# Prediction using gini
	y_pred_gini = prediction(X_test, clf_gini)
	cal_accuracy(y_test, y_pred_gini)
	
	print("Results Using Entropy:")
	# Prediction using entropy
	y_pred_entropy = prediction(X_test, clf_entropy)
	cal_accuracy(y_test, y_pred_entropy)
	
	
# Calling main function
if __name__=="__main__":
	main()


##KNN Classification


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test =  train_test_split(X, Y, test_size = 0.30,random_state = 17)
#X_train, Y_test = train_test_split(df, test_size = 0.20, random_state= 17)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.fit_transform(X_test)
# X_train =  scaler.transform(X_train)
# X_test  =  scaler.transform(X_test)

In [ ]:
from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(Y_train)


print(utils.multiclass.type_of_target(Y_train))

print(utils.multiclass.type_of_target(Y_train.astype('int')))


print(utils.multiclass.type_of_target(encoded))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,accuracy_score
import seaborn as sns
def get_knn(X_train,Y_train,X_test,Y_test,n_neighbors):
  neigh = KNeighborsClassifier(n_neighbors=n_neighbors)
  neigh.fit(X_train, Y_train)
  accuracy = accuracy_score(neigh.predict(X_test),Y_test)
  print('Accuracy:',accuracy,'\n\n')
  print(classification_report(neigh.predict(X_test),Y_test))
  return accuracy

In [ ]:
accuracy_list = []
n_neighbor_list = []
accuracy_std_list = []
for i in range(1,11):
  print("KNN for n= ",i," after flattening - (1540 * 8 x 33) :\n")
  accuracy = get_knn(X_train,Y_train,X_test,Y_test,n_neighbors=i)
  print("KNN after standardization  for n= ",i,":\n")
  accuracy_std = get_knn(X_train_std,Y_train,X_test_std,Y_test,n_neighbors=i)
  accuracy_list.append(accuracy)
  n_neighbor_list.append(i)
  accuracy_std_list.append(accuracy_std)